# Unsupervised Learning

Unsupervised learning describes a collection of techniques that attempt to find structure in data.

Examples include:

* __Clustering__: attempts to divide data into groups based off a definition of similarity, such as distance in univariate or multidimensional space.
* __Dimensionality Reduction__: Attempts to encode high dimensional data (e.g. tens of thousands of variables) into a smaller set of variables

This is in contrast to supervised learning, where, given an input $x$ and output $y$ we attempt to learn a function $f(x)$ = $y$, where $x$ and $y$ can be scalars (numbers) , vectors (lines of numbers), or matrices (2D array of numbers). In unsupervised learning, we are only given $x$ and no output, and the goal is to find structure in a collection of data points.

Here, we'll go over 2 Techniques: dimensionality reduction and clustering.

## Dimensionality Reduction

Often we can deal with datasets that exist in high dimensional spaces (e.g $d$ = 10,000, where $d$ is the number of variables in the data). The breast cancer dataset we shall explore has 30 dimensions for 'input data' features  plus 1 for 'target' label (i.e. whether or not the example is benign or malignant). Not only is it impossible for people to visualize data beyond 3 dimensions, but when we use supervised learning techniques such as K-nearest neighbor classification or an unsupervised method such as K-means clustering, the assumption that similar datapoints are closer to one another than dissimilar points in terms of Euclidean distance is violated<sup>1</sup>. The violation of this assumption is called the _curse of dimensionality_. A further discussion of this topic can be found [here](https://www.cs.cornell.edu/courses/cs4780/2018fa/lectures/lecturenote02_kNN.html).

What dimensionality reduction can accomplish is to pick up neat properties of your data in high dimensional space and express them in a much lower dimensional space. This allows an automated fashion of finding those neat properties while also expressing data in a way better suited for downstream applications such as clustering or classification. Examples include Princpal Components Analysis (PCA), Factor Analysis, Locally Linear Embedding, and more. Today we'll go over PCA.

1. For 2D and 3D data, Euclidean distance is just the length of a line drawn between two points $p$ and $q$ if you plotted them in 2 or 3D, and its formula is $\sqrt{\sum_{d=1}^D (p_d - q_d)^2} $, where $d$ denotes an individual variable and $D$ is the total number of variables. Here's an example in 2D from Wikipedia:

![Euclidean Distance in 2D (Wikipedia)](https://upload.wikimedia.org/wikipedia/commons/5/55/Euclidean_distance_2d.svg)

### Overview of PCA:

Without diving into linear algebra, what PCA tries to do at the end of the day is express the data as weighted sums of the data's variables.

The procedure is as follows:

1. Standard Normalize Data. Part of the reason we do this is so the procedure doesn't give too much weight to certain variables based on certain variables having a wider range of values.

    Formula is as follows: $b_{id} = (x_{id} - \bar{x_d})/\sigma_d$, where $i$ denotes sample, $d$ denotes variable,  $\bar{x_d}$ denotes mean for the variable among all $x_{id}$, and $\sigma_d$ denotes the standard deviation for variable d

2. calculate a variable $v_1$ = $\sum_{d=1}^{D} a_d b_d$ such that $Var[v_1]$ (variance of the new variable) is maximized, with the constraint that $\sum_{d=1}^{D} a_d^2$ = 1

3. repeat step 2, but subject to the constraint that the new variable, $v_2$ is uncorrelated with $v_1$, i.e correlation between the two variables = 0.

4. Repeat step 3, again and again, subject to the constraint that any new variable $v_k$ computed for iteration $l$ is not correlated with any variables $v_{k'}$ where $k' < k$ (i.e not correlated with any previously computed variables).

There's an upper limit on the amount of PCs that can be computed, but we're not discussing that today as it requires knowledge of linear algebra.

For a more in depth discussion of PCA, here's a reference you can look into:

* [Stanford Engineering Everywhere, CS229 PCA Notes](https://see.stanford.edu/materials/aimlcs229/cs229-notes10.pdf)

Let's dive into the data!

### PCA on Breast Cancer Dataset (TODO: Implement)



In [1]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()

In [6]:
data.feature_names

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

In [4]:
data.target_names

array(['malignant', 'benign'], dtype='<U9')

## Clustering (Note, skip hierarchical for time; TODO: Implement)

### Overview

Suppose you want to discover if there are groups of samples that are similar to one another, but don't know where to start? Manually examining numbers and grouping things together is not something people are great at, unless it's points in 2D and 3D space. Even then, you want to have a definition of how you got those groups. Clustering refers to automated procedures by which we can find groups given a bunch of samples defined by numbers.

### Kmeans

One example of clustering is kmeans, which attempts to find groups of samples where samples in the same group are closer to one another than they are to other samples.

The function it tries to minimize is as follows:

$\sum_{i=1}^{N}\sum_{k=1}^K r_{ik} \cdot dist(x_i, \mu_k)^2$

where $x_i$ is a datapoint, $r_{ik}$ = 1 if $x_i$ is in cluster $k$ and = 0 otherwise, and $dist(x_i, \mu_k)$ is the Euclidean distance between the datapoint and the mean of all datapoints in cluster k.

More details on kmeans algorithm [here](https://see.stanford.edu/materials/aimlcs229/cs229-notes7a.pdf).

We can try clustering with a varying number of clusters. For this exercise, we're going to use the first 2 PCs we found in the breast cancer dataset, as it will be easier to visualize what kmeans is doing.

Let's try k = 2:

Notice how the data's been partitioned into two clusters of samples that are close to one another?

Let's try k = 3:

Now we have 3 groups of samples that looks close to one another.

### Side Notes on Kmeans

Kmeans starts with randomly assigned clusters, and tries to improve cluster assignment in an iterative process. We set a seed for a random number generator which ensures that, given a particular random seed, we get the same clustering result every time. Different start points can result in different solutions. If you really want to be rigorous about checking if the solutions that come out don't vary too much given different start points, you can run kmeans 100-1000 times with different seeds and calculate the similarity of solutions with the [adjusted rand index](https://en.wikipedia.org/wiki/Rand_index#Adjusted_Rand_index).

Kmeans works best if data is distributed roughly spherically, i.e. the density of points emanating from the center of a given group doesn't change that much depending on the direction outward from the center of the cluster. Kmeans also doesn't work well when groups of samples have a curved distribution. Examples of how violations of this assumption can lead to bad clustering are given [here](https://scikit-learn.org/stable/modules/clustering.html)

#### Choosing the number of clusters

The question then arises: How many clusters is the best number? If we choose too few, we might not be capturing all of the cool structure in the data. 

We can use the silhouette score, which compares the the distance between samples in the same cluster to samples in the nearest cluster (more on the silhouette score [here](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html).  Ideally, we want to maximize this score across all clusters, so we'll take the average silhouette score across all samples and make this the function we want to maximize.

Let's see both the clusters that come out and the resulting scores

#### Exploring Properties of Clusters

We can do a quick visualization of how certain variables distribute in our clusters. (do visualization of 2-3 variables)

If we want to be confident that these differences are 'real', we do statistical tests. Here, we'll compare (variable) between cluster 1 and all cluster 2.

## Summary:

We did an initial data exploration of a breast cancer dataset with PCA, and learned how clustering can automatically group similar samples together. Now it's your turn to apply what you've learned to your own data and see if you can find any cool insights!